## Структура базы данных, к которой приводим

![Итоговый вид со всеми связями после нормализации](db_tables.png)


## Нормализация к первой форме (1НФ)

<details>
<summary>Развернуть/свернуть</summary>

Изначальная таблица с данными уже соответствует первой нормальной форме, так в каждом столбце файла все атрибуты имеют скалярное значение, то есть их нельзя разложить на более простые составляющие.

Пример строки данных листа `transaction`:  
(начало)
| transaction_id | product_id | customer_id | transaction_date | online_order | order_status | brand |
|----------------|------------|-------------|------------------|--------------|--------------|-------|
| 1              | 2          | 2950        | 25/2/2017        | False        | Approved     | Solex |

(продолжение)
| product_line | product_class | product_size | list_price | standard_cost |
|--------------|---------------|--------------|------------|---------------|
| Standard     | medium        | medium       | 71.49      | 53.62         |

Пример строки данных листа `customer`:  

(начало)
| customer_id | first_name | last_name | gender | DOB         | job_title           | job_industry_category | wealth_segment |
|-------------|------------|-----------|--------|-------------|---------------------|-----------------------|----------------|
| 2950        | Kristos    | Anthony   | Male   | 1955-01-11  | Software Engineer I | Financial Services    | Mass Customer  |

(продолжение)
| deceased_indicator | owns_car | address              | postcode | state          | country    | property_valuation |
|--------------------|----------|----------------------|----------|----------------|------------|--------------------|
| N                  | Yes      | 3 New Castle Terrace | 3032     | VIC            | Australia  | 8                  |

</details>

## Нормализация ко второй форме (2НФ)

<details>
<summary>Развернуть/свернуть</summary>

Требует, чтобы данные одновременно соответствовали 1НФ (+) и каждый атрибут зависил только от первичного ключа (-).  
1. Лист `transaction`:
- такие поля, как `brand` `product_line` `product_class` `product_size` `list_price` `standard_cost`, зависят от `product_id`, а не от `transaction_id`
- необходимо вынести зависимые поля в отдельный лист `product`. Тогда таблица `product` будет иметь следующий вид:  

  | product_id | brand | product_line | product_class | product_size | list_price | standard_cost |
  |------------|-------|--------------|---------------|--------------|------------|---------------|
  | 2          | Solex | Standard     | medium        | medium       | 71.49      | 53.62         |

2. Лист `customer`:  
- здесь существует один первичный ключ, все поля так или иначе относятся к представлению пользователя, таблица `customer` на данном этапе будет соответствовать данным строки из листа.

После внесения изменений, таблица `transaction` будет иметь вид:  

| transaction_id | product_id | customer_id | transaction_date | online_order | order_status |
|----------------|------------|-------------|------------------|--------------|--------------|
| 1              | 2          | 2950        | 2/25/2017        | False        | Approved     |

</details>

## Нормализация к третьей форме (3НФ)

<details>
<summary>Развернуть/свернуть</summary>

Требует, чтобы данные одновременно соответствовали 2НФ (+) и любой атрибут должен зависеть только от первичного ключа, то есть должна отстутствовать транзитивная связь (-).  

1. Лист `transaction`:  
- транзитивные связи отсутствуют
- если предположить, что поле `order_status` используется как `доставили/не доставили` без расширения до `В пути` или `Ожидает оплаты`, то сделаем `Enum`-таблицу:

  `Enum`
  | order_status |
  |--------------|
  | Approved     |
  | Cancelled    |


2. Лист `product`:  
- здесь нет транзитивных связей, все поля напрямую принадлежат сущности `product`
- однако для полей `brand` `product_line` `product_class` `product_size` можно создать справочные таблицы для более гибкого и централизованного изменения данных о товаре. Например, для поля `brand` изменится название фирмы с `Solex` на `Solex & CO`, тогда нам не придется проходить по всем товарам, а потребуется внести правки только для конкретной записи в таблице `brand`
- конечно, многое зависит от бизнес-требований, есть ли необходимость в том, чтобы плодить множество связанных таблиц или достаточно сделать отдельную с `Enum` жесто заданными, например, размерами `product_size`. Насколько мы уверены, что значения не будут меняться?
- после изучения документации postgres и различных статей, можно сделать вывод, что использование `Enum-таблиц` в некотором смысле считается антипаттерном; использование справочных таблиц работает быстрее, чем получение одной большой строки данных, так как, во-первых, современные движки очень хорошо JOIN'ят небольшие таблицы, во-вторых, за счет механизва индексации и, в-третьих, кеширование часто запрашиваемых данных.
- тогда, таблицы и их связь будет выглядеть следующим образом:

  `product`
  | product_id | brand_id | product_line_id | product_class_id | product_size_id | list_price | standard_cost |
  |------------|----------|-----------------|------------------|-----------------|------------|---------------|
  | 2          | 1        | 2               | 4                | 2               | 71.49      | 53.62         |

  `brand`
  | brand_id   | brand |
  |------------|-------|
  | 1          | Solex |

  `product_line`
  | product_line_id   | product_line |
  |-------------------|--------------|
  | 2                 | Standard     |

  `product_class`
  | product_class_id   | product_class |
  |--------------------|---------------|
  | 4                  | medium        |

  `product_size`
  | product_size_id   | product_size |
  |-------------------|--------------|
  | 2                 | medium       |

3. Лист `customer`:
- здесь явно требуется вынести в отдельную сущность всё, что связано с адресом в отдельную таблицу `address`
- потенциально изменяемые или дополняемые поля можно вынести в отдельные таблицы `state` и `country`. Тогда:

  `address`
  | address_id | state_id | country_id   | address              | postcode |
  |------------|----------|--------------|----------------------|----------|
  | 1          | 1        | 1            | 3 New Castle Terrace | 3032     |

  `state`
  | state_id   | state     |
  |------------|-----------|
  | 1          | VIC       |

  `country`
  | country_id   | country    |
  |--------------|------------|
  | 1            | Australia  |
-
- что касается полей `job_title` и `job_industry_category`, то они, скорее всего, относятся к отдельной сущности `job`:

  `job`
  | job_id     | job_industry_category_id | job_title           |
  |------------|--------------------------|---------------------|
  | 1          | 1                        | Software Engineer I |

  `job_industry_category`
  | job_industry_category_id   | category_name            |
  |----------------------------|--------------------------|
  | 1                          | Financial Services       |
-
- для полей `gender` `deceased_indicator` `owns_car` можно выделить в `Enum`-таблицы, здесь это оправдано, так как по смыслу они не должны меняться и жестко фиксированы. Поле `wealth_segment` может дополняться, так что выделим под это списочную таблицу:

  `Enum`
  | gender            | deceased_indicator | owns_car           |
  |-------------------|--------------------|--------------------|
  | F                 | N                  | Yes                |
  | Male              | Y                  | No                 |
  | Female            |
  | U                 |
  | Femal             |
  | M                 |

  `wealth_segment`  
  | wealth_segment_id   | segment_name        |
  |---------------------|---------------------|
  | 1                   | Mass Customer       |
-
- тогда итоговая таблица `customer` будет иметь вид:

  `customer`  
  (начало)
  | customer_id | address_id | wealth_segment_id | job_id | first_name | last_name | gender |
  |-------------|------------|-------------------|--------|------------|-----------|--------|
  | 2950        | 1          | 1                 | 1      | Kristos    | Anthony   | Male   |

  (продолжение)
  | DOB         | deceased_indicator | owns_car | roperty_valuation  |
  |-------------|--------------------|----------|--------------------|
  | 1955-01-11  | N                  | Yes      | 8                  |

</details>


## Загрузка данных

Так как делать это посредством импорта csv-файла в `DBeaver` не очень интересно, то воспользуюсь адаптером для работы с БД напрямую через драйвер и объект `Connection`. Работа выполнялась с использованием [документации](https://www.psycopg.org/psycopg3/docs/index.html).

In [1]:
%pip install "psycopg[binary]" pandas openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 826.7 kB/s eta 0:00:00a 0:00:01
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached psycopg-3.2.12-py3-none-any.whl.metadata (4.5 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached psycopg_binary-3.2.12-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (2.9 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 5.2 MB/s eta 0:00:00
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached psycopg_binary-3.2.12-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (4.4 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 31.5 MB/s eta 0:00:0000:0100:01
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 20.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 41.7 MB/s eta 0:00:00
Using cached typing_

In [1]:
import psycopg
import pandas as pd

### Подготовка фреймов с данными

In [2]:
sheets = pd.read_excel(
    "customer_and_transaction.xlsx",
    sheet_name=None,
    na_filter=False,
)

transaction_df = sheets["transaction"]
product_df = sheets["product"]

brand_df = sheets["brand"]
product_line_df = sheets["product_line"]
product_class_df = sheets["product_class"]
product_size_df = sheets["product_size"]

customer_df = sheets["customer"]
job_df = sheets["job"]
wealth_segment_df = sheets["wealth_segment"]
job_industry_category_df = sheets["job_industry_category"]

address_df = sheets["address"]
state_df = sheets["state"]
country_df = sheets["country"]

enums_df = sheets["enums"]
order_status = enums_df["order_status"]
gender = enums_df["gender"]
owns_car = enums_df["owns_car"]
deceased_indicator = enums_df["deceased_indicator"]

### Создадим объект подключения к БД

In [3]:
connection = psycopg.connect(
    dbname="test",
    user="postgres",
    password="1234qaz",
    host="localhost",
    port="5432",
)

cur = connection.cursor()

### Работа с данными таблицы `product`

В изначальном файле с данными есть подозрительные строки, которые выглядят как "битые". Только у таких записей отсутствуют поля `brand` `product_line` `product_class` `product_size` `standard_cost`. Можно предположить, что для нормальной записи эти поля не должны быть пустыми, поэтому для связанных полей и полей с ценой я решил сделать `NOT NULL` (хотя на практике любое из указанных связанных полей может быть и `NULL`, так как, например, размер у товара может отсутствовать в принципе).

(пример)
| product_id | brand | product_line | product_class | product_size | list_price | standard_cost |
|------------|-------|--------------|---------------|--------------|------------|---------------|
| 0          |       |              |               |              | 1656.86    |               |
| 0          |       |              |               |              | 850.89     |               |
| 0          |       |              |               |              | 710.59     |               |

In [4]:
cur.execute("""
    CREATE TABLE IF NOT EXISTS brand (
        brand_id serial PRIMARY KEY,
        brand_name VARCHAR(255) NOT NULL
    )
""")

cur.execute("""
    CREATE TABLE IF NOT EXISTS product_line (
        product_line_id serial PRIMARY KEY,
        line_name VARCHAR(255) NOT NULL
    )
""")

cur.execute("""
    CREATE TABLE IF NOT EXISTS product_class (
        product_class_id serial PRIMARY KEY,
        class_name VARCHAR(255) NOT NULL
    )
""")

cur.execute("""
    CREATE TABLE IF NOT EXISTS product_size (
        product_size_id serial PRIMARY KEY,
        size_name VARCHAR(255) NOT NULL
    )
""")

cur.execute("""
    CREATE TABLE IF NOT EXISTS product (
        product_id INTEGER PRIMARY KEY,
        brand_id INTEGER NOT NULL REFERENCES brand(brand_id),
        product_line_id INTEGER NOT NULL REFERENCES product_line(product_line_id),
        product_class_id INTEGER NOT NULL REFERENCES product_class(product_class_id),
        product_size_id INTEGER NOT NULL REFERENCES product_size(product_size_id),
        list_price NUMERIC NOT NULL,
        standard_cost NUMERIC NOT NULL
    )
""")

brand_insert_query = """
    INSERT INTO brand (brand_name) VALUES (%s)
"""

product_line_insert_query = """
    INSERT INTO product_line (line_name) VALUES (%s)
"""

product_class_insert_query = """
    INSERT INTO product_class (class_name) VALUES (%s)
"""

product_size_insert_query = """
    INSERT INTO product_size (size_name) VALUES (%s)
"""

cur.executemany(
    brand_insert_query,
    [tuple(value) for value in brand_df.values],
)

cur.executemany(
    product_line_insert_query,
    [tuple(value) for value in product_line_df.values],
)

cur.executemany(
    product_class_insert_query,
    [tuple(value) for value in product_class_df.values],
)

cur.executemany(
    product_size_insert_query,
    [tuple(value) for value in product_size_df.values],
)

product_insert_query = """
    INSERT INTO product (
        product_id,
        brand_id,
        product_line_id,
        product_class_id,
        product_size_id,
        list_price,
        standard_cost
    ) VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

cur.executemany(
    product_insert_query,
    [tuple(value) for value in product_df.values]
)

cur.execute("""
    SELECT * FROM product
""")

for i, row in enumerate(cur.fetchall()):
    print(f"Вывод строки #{i}: {row}")

connection.commit()


Вывод строки #0: (2, 1, 1, 1, 1, Decimal('71.49'), Decimal('53.62'))
Вывод строки #1: (3, 2, 1, 1, 2, Decimal('2091.47'), Decimal('388.92'))
Вывод строки #2: (37, 3, 1, 2, 1, Decimal('1793.43'), Decimal('248.82'))
Вывод строки #3: (88, 4, 1, 1, 1, Decimal('1198.46'), Decimal('381.1'))
Вывод строки #4: (78, 5, 1, 1, 2, Decimal('1765.3'), Decimal('709.48'))
Вывод строки #5: (25, 5, 2, 1, 1, Decimal('1538.99'), Decimal('829.65'))
Вывод строки #6: (22, 6, 1, 1, 1, Decimal('60.34'), Decimal('45.26'))
Вывод строки #7: (15, 6, 1, 1, 1, Decimal('1292.84'), Decimal('13.44'))
Вывод строки #8: (67, 1, 1, 1, 2, Decimal('1071.23'), Decimal('380.74'))
Вывод строки #9: (12, 6, 1, 1, 1, Decimal('1231.15'), Decimal('161.6'))


### Работа с данными таблицы `address`

В изначальном файле с данными поля, относящиеся к адресу, не содержат пустых значений, поэтому все поля необходимы (например, для API доставки).

In [5]:
cur.execute("""
    CREATE TABLE IF NOT EXISTS country (
        country_id serial PRIMARY KEY,
        country_name VARCHAR(255) NOT NULL
    )
""")

cur.execute("""
    CREATE TABLE IF NOT EXISTS state (
        state_id serial PRIMARY KEY,
        state_name VARCHAR(255) NOT NULL
    )
""")

cur.execute("""
    CREATE TABLE IF NOT EXISTS address (
        address_id serial PRIMARY KEY,
        country_id INTEGER NOT NULL REFERENCES country(country_id),
        state_id INTEGER NOT NULL REFERENCES state(state_id),
        address VARCHAR(255) NOT NULL,
        postcode INTEGER NOT NULL
    )
""")

country_insert_query = """
    INSERT INTO country (country_name) VALUES (%s)
"""
cur.executemany(
    country_insert_query,
    [tuple(value) for value in country_df.values],
)

state_insert_query = """
    INSERT INTO state (state_name) VALUES (%s)
"""
cur.executemany(
    state_insert_query,
    [tuple(value) for value in state_df.values],
)

address_insert_query = """
    INSERT INTO address (
        country_id,
        state_id,
        address,
        postcode
    ) VALUES (%s, %s, %s, %s)
"""
cur.executemany(
    address_insert_query,
    [tuple(value) for value in address_df.values]
)

cur.execute("""
    SELECT * FROM address
""")

for i, row in enumerate(cur.fetchall()):
    print(f"Вывод строки #{i}: {row}")

connection.commit()

Вывод строки #0: (1, 1, 1, '060 Morning Avenue', 2016)
Вывод строки #1: (2, 1, 1, '6 Meadow Vale Court', 2153)
Вывод строки #2: (3, 1, 2, '0 Holy Cross Court', 4211)
Вывод строки #3: (4, 1, 1, '17979 Del Mar Point', 2448)
Вывод строки #4: (5, 1, 3, '9 Oakridge Court', 3216)
Вывод строки #5: (6, 1, 1, '4 Delaware Trail', 2210)
Вывод строки #6: (7, 1, 1, '49 Londonderry Lane', 2650)
Вывод строки #7: (8, 1, 1, '97736 7th Trail', 2023)
Вывод строки #8: (9, 1, 3, '93405 Ludington Park', 3044)
Вывод строки #9: (10, 1, 2, '44339 Golden Leaf Alley', 4557)


### Работа с данными таблицы `job`

В изначальных данных, поля `job_title` может отсутствовать, но не `job_industry_category`.

In [6]:
cur.execute("""
    CREATE TABLE IF NOT EXISTS job_industry_category (
        job_industry_category_id serial PRIMARY KEY,
        category_name VARCHAR(255) NOT NULL
    )
""")

cur.execute("""
    CREATE TABLE IF NOT EXISTS job (
        job_id serial PRIMARY KEY,
        job_industry_category_id INTEGER NOT NULL REFERENCES job_industry_category(job_industry_category_id),
        job_title VARCHAR(255)
    )
""")

job_industry_category_insert_query = """
    INSERT INTO job_industry_category (category_name) VALUES (%s)
"""
cur.executemany(
    job_industry_category_insert_query,
    [tuple(value) for value in job_industry_category_df.values],
)

job_insert_query = """
    INSERT INTO job (
        job_industry_category_id,
        job_title
    ) VALUES (%s, %s)
"""
cur.executemany(
    job_insert_query,
    [tuple(value) for value in job_df.values],
)

cur.execute("""
    SELECT * FROM job
""")

for i, row in enumerate(cur.fetchall()):
    print(f"Вывод строки #{i}: {row}")

connection.commit()


Вывод строки #0: (1, 1, 'Executive Secretary')
Вывод строки #1: (2, 2, 'Administrative Officer')
Вывод строки #2: (3, 3, 'Recruiting Manager')
Вывод строки #3: (4, 4, '')
Вывод строки #4: (5, 5, 'Senior Editor')
Вывод строки #5: (6, 6, '')
Вывод строки #6: (7, 2, '')
Вывод строки #7: (8, 5, 'Media Manager I')
Вывод строки #8: (9, 7, 'Business Systems Development Analyst')
Вывод строки #9: (10, 2, 'Senior Quality Engineer')


### Работа с данными таблицы `customer`

Исходя из изначальных данных, поля `last_name` и `dob` могут быть пустыми. Неизвестно, какое требование предоставляется для подтвеждения транзакции, какие данные необходимы, но так как почти все поля заполнены, то можно предположить, что это обязательное условие. Следовательно, для таблицы `customer` все поля, кроме `last_name` и `dob` являются не пустыми.

In [7]:
# создание Enum-таблиц

gender_values = ", ".join(f"'{value}'" for value in gender.values)
cur.execute(f"""
    CREATE TYPE gender AS ENUM ({gender_values})
""")

deceased_indicator_values = ", ".join(f"'{value}'" for value in deceased_indicator.values if value)
cur.execute(f"""
    CREATE TYPE deceased_indicator AS ENUM ({deceased_indicator_values})
""")

owns_car_values = ", ".join(f"'{value}'" for value in owns_car.values if value)
cur.execute(f"""
    CREATE TYPE owns_car AS ENUM ({owns_car_values})
""")

connection.commit()

In [8]:
# создание основной таблицы с данными

cur.execute("""
    CREATE TABLE IF NOT EXISTS wealth_segment (
        wealth_segment_id serial PRIMARY KEY,
        segment_name VARCHAR(255) NOT NULL
    )
""")

cur.execute("""
    CREATE TABLE IF NOT EXISTS customer (
        customer_id INTEGER PRIMARY KEY,
        address_id INTEGER NOT NULL REFERENCES address(address_id),
        job_id INTEGER NOT NULL REFERENCES job(job_id),
        wealth_segment_id INTEGER NOT NULL REFERENCES wealth_segment(wealth_segment_id),
        first_name VARCHAR(255) NOT NULL,
        last_name VARCHAR(255),
        gender gender NOT NULL,
        dob DATE,
        deceased_indicator deceased_indicator NOT NULL,
        owns_car owns_car NOT NULL,
        property_valuation INTEGER NOT NULL
    )
""")

wealth_segment_query = """
    INSERT INTO wealth_segment (segment_name) VALUES (%s)
"""
cur.executemany(
    wealth_segment_query,
    [tuple(value) for value in wealth_segment_df.values],
)

customer_insert_query = """
    INSERT INTO customer (
        customer_id,
        address_id,
        job_id,
        wealth_segment_id,
        first_name,
        last_name,
        gender,
        dob,
        deceased_indicator,
        owns_car,
        property_valuation
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
cur.executemany(
    customer_insert_query,
    [tuple(value) for value in customer_df.values],
)

cur.execute("""
    SELECT * FROM customer
""")

for i, row in enumerate(cur.fetchall()):
    print(f"Вывод строки #{i}: {row}")

connection.commit()

Вывод строки #0: (2950, 1, 1, 1, 'Laraine', 'Medendorp', 'F', datetime.date(1953, 10, 12), 'N', 'Yes', 10)
Вывод строки #1: (3120, 2, 2, 1, 'Eli', 'Bockman', 'Male', datetime.date(1980, 12, 16), 'N', 'Yes', 10)
Вывод строки #2: (402, 3, 3, 1, 'Arlin', 'Dearle', 'Male', datetime.date(1954, 1, 20), 'N', 'Yes', 9)
Вывод строки #3: (3135, 4, 4, 1, 'Talbot', '', 'Male', datetime.date(1961, 10, 3), 'N', 'No', 4)
Вывод строки #4: (787, 5, 5, 2, 'Sheila-kathryn', 'Calton', 'Female', datetime.date(1977, 5, 13), 'N', 'Yes', 9)
Вывод строки #5: (2339, 6, 6, 3, 'Curr', 'Duckhouse', 'Male', datetime.date(1966, 9, 16), 'N', 'Yes', 9)
Вывод строки #6: (1542, 7, 7, 2, 'Fina', 'Merali', 'Female', datetime.date(1976, 2, 23), 'N', 'Yes', 4)
Вывод строки #7: (2459, 8, 8, 1, 'Rod', 'Inder', 'Male', datetime.date(1962, 3, 30), 'N', 'No', 12)
Вывод строки #8: (1305, 9, 9, 2, 'Mala', 'Lind', 'Female', datetime.date(1973, 3, 10), 'N', 'Yes', 8)
Вывод строки #9: (3262, 10, 10, 1, 'Fiorenze', 'Birdall', 'Female'

### Создание таблицы `transaction`

Исходя из изначальных данных, поле `online_order` может быть пустым, а все остальные - нет.

In [9]:
# создание Enum-таблицы

order_status_values = ", ".join(f"'{value}'" for value in order_status.values if value)
cur.execute(f"""
    CREATE TYPE order_status AS ENUM ({order_status_values})
""")

connection.commit()

In [10]:
# создание основной таблицы с данными

cur.execute("""
    CREATE TABLE IF NOT EXISTS transaction (
        transaction_id serial PRIMARY KEY,
        product_id INTEGER NOT NULL REFERENCES product(product_id),
        customer_id INTEGER NOT NULL REFERENCES customer(customer_id),
        transaction_date DATE NOT NULL,
        online_order BOOL,
        order_status order_status NOT NULL
    )
""")

transaction_insert_query = """
    INSERT INTO transaction (
        product_id,
        customer_id,
        transaction_date,
        online_order,
        order_status
    ) VALUES (%s, %s, %s, %s, %s)
"""
cur.executemany(
    transaction_insert_query,
    [tuple(value) for value in transaction_df.values],
)

cur.execute("""
    SELECT * FROM transaction
""")

for i, row in enumerate(cur.fetchall()):
    print(f"Вывод строки #{i}: {row}")

connection.commit()

connection.close()


Вывод строки #0: (1, 2, 2950, datetime.date(2017, 2, 25), False, 'Approved')
Вывод строки #1: (2, 3, 3120, datetime.date(2017, 5, 21), True, 'Approved')
Вывод строки #2: (3, 37, 402, datetime.date(2017, 10, 16), False, 'Approved')
Вывод строки #3: (4, 88, 3135, datetime.date(2017, 8, 31), False, 'Approved')
Вывод строки #4: (5, 78, 787, datetime.date(2017, 10, 1), True, 'Approved')
Вывод строки #5: (6, 25, 2339, datetime.date(2017, 3, 8), True, 'Approved')
Вывод строки #6: (7, 22, 1542, datetime.date(2017, 4, 21), True, 'Approved')
Вывод строки #7: (8, 15, 2459, datetime.date(2017, 7, 15), False, 'Approved')
Вывод строки #8: (9, 67, 1305, datetime.date(2017, 8, 10), False, 'Approved')
Вывод строки #9: (10, 12, 3262, datetime.date(2017, 8, 30), True, 'Approved')
